<a href="https://colab.research.google.com/github/lloppy/urfu_neironki-5sem/blob/main/server_chemistry_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

добавим импорты

In [10]:
!pip install fastapi uvicorn[standard] nest-asyncio pyngrok scikit-learn joblib

import nest_asyncio
from pyngrok import ngrok
from pyngrok.conf import PyngrokConfig
import joblib
import numpy as np
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, HTTPException

In [11]:
# 1. Настройка ngrok с вашим токеном
ngrok.set_auth_token("2wSJ5VaAqlgK9jjKEjqXJIzJjq0_iyrjVymxXRqBZi4Q7pjy")

# 2. Инициализация FastAPI
app = FastAPI()

# Разрешить CORS (для тестов)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# 3. Загрузка модели (пример)
# Загрузите файл модели через левую панель Colab
model = joblib.load("gb_model.joblib")
model

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

GradientBoostingClassifier(learning_rate=0.2, max_features='log2',
                           min_samples_split=5, n_estimators=400,
                           random_state=42)

In [12]:
# 4. API Endpoint
@app.post("/predict")
async def predict(features: list[float]):
    try:
        input_array = np.array(features).reshape(1, -1)
        prediction = model.predict_proba(input_array)[0, 1]
        return {
            "status": "success",
            "prediction": float(prediction),
            "model_type": str(type(model))
        }
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# 5. Запуск сервера
if __name__ == "__main__":
    nest_asyncio.apply()

    # Открываем туннель
    public_url = ngrok.connect(8000).public_url
    print("\n🔥 Сервер доступен по URL:", public_url)
    print("📚 Документация API:", f"{public_url}/docs\n")

    # Запускаем FastAPI
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


🔥 Сервер доступен по URL: https://b991-34-75-246-147.ngrok-free.app
📚 Документация API: https://b991-34-75-246-147.ngrok-free.app/docs



INFO:     Started server process [570]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     95.164.51.20:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     95.164.51.20:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     5.189.111.215:0 - "POST /predict HTTP/1.1" 422 Unprocessable Entity


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [570]
